In [8]:
#This notebook is for visualizing calibrated neural likelihood surfaces for the evaluation data.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [9]:
#Load the data
version = "final_version"
image_size = 25
image_name = str(image_size) + "_by_" + str(image_size)
local_folder = "/home/juliatest/Desktop/likelihood_free_inference/neural_likelihood/brown_resnick"
fields_file = (local_folder + "/evaluate_nn/produce_neural_likelihood_surfaces/data/" + image_name + "/" + version + 
"/calibrated/single/reps/200/calibrated_neural_likelihood_surfaces_10_by_10_density_" + image_name + "_image_200.npy")
fields = np.load(fields_file)

number_of_parameters = 100
number_of_reps = 200
true_params = [(.2*j, .2*i) for i in range(1,11) for j in range(1,11)]
possible_ranges = [.05*i for i in range(1, 41)]
possible_smooths = [.05*i for i in range(1, 41)]

#first column is range and second is smooth
parameter_matrix = np.load((local_folder + "/evaluate_nn/generate_data/data/" + image_name + 
                            "/single/reps/200/evaluation_parameters_10_by_10_density_" + image_name + "_200.npy"))

In [10]:
#Save a png file of the calibrated neural likelihood surface.
#function parameters:
    #calibrated_neural_likelihood_surface: (numpy matrix) calibrated neural likelihood surface for a realization of the GP from
    # evaluation data
    #true_range: the range value which generated the realization for this calibrated neural likelihood surface
    #true_smooth: the smooth value which generated the realization for this calibrated neural likelihood surface
    #possible_ranges: range values on the parameter grid
    #possible_smooths: smooth values on the parameter grid
    #C: the critical value for around 90 percent coverage according to Wilk's Thrm (multiple realizations) and the range of the
    #color scale (ranges from max of the field to max of the field minus C)
    #rep_number: the replication number that identifies the realization which produced the given calibrated neural likelihood surface
def visualize_calibrated_neural_likelihood_surface(calibrated_neural_likelihood_surface, possible_ranges, possible_smooths,
                                                    true_range, true_smooth, rep_number, C):

    fig, ax = plt.subplots(figsize = (10,10))
    x = np.linspace(.05, 2, len(possible_ranges))
    y = np.linspace(.05, 2, len(possible_smooths))
    X, Y = np.meshgrid(x, y)
    Z = calibrated_neural_likelihood_surface
    Z[Z==0] = np.min(Z[Z!=0])
    Z = np.log(Z)
    Z = Z.reshape((40, 40))
    max_indices = np.unravel_index(np.argmax(Z, axis=None), Z.shape)
    max_range = possible_ranges[max_indices[0]]
    max_smooth = possible_smooths[max_indices[1]]
    ax.contourf(X, Y, Z, vmin = (np.amax(Z) - C), vmax = np.amax(Z), levels = 12000)
    ax.scatter(true_smooth, true_range, s = 600, marker = "*", c = "black")
    ax.scatter(max_smooth, max_range, s = 600, marker = "o", c= "red")
    legend_elements = [Line2D([0], [0], marker='*', color='w', label='True',
                          markerfacecolor='black', markersize=40), 
                          Line2D([0], [0], marker='o', color='w', label='Estimate',
                          markerfacecolor='red', markersize=30)]
    ax.legend(handles = legend_elements, facecolor='white', framealpha=1, fontsize="25")
    ax.set_xlabel("smooth", fontsize = 40)
    ax.set_ylabel("range", fontsize = 40)
    plt.yticks(fontsize=20)
    plt.xticks(fontsize =20)
    plt.tight_layout()
    fig_name = (local_folder + "/evaluate_nn/visualizations/visualize_surfaces/surfaces/" + image_name + 
                "/neural_likelihood/" + version + "/calibrated/single/reps/200/"
                 + str(C) + "_calibrated_neural_likelihood_surface_smooth_" + str(round(true_smooth, 2))
     + "_range_" + str(round(true_range, 2)) + "_rep_" + str(rep_number) + ".png")
    #plt.savefig(fig_name)
    plt.show()

In [ ]:
possible_smooths = [.05*i for i in range(1, 41)]
possible_ranges = [.05*i for i in range(1, 41)]
C = 10
indices = [11, 13, 15, 17, 31, 33, 35, 37, 51, 53, 55, 57, 71, 73, 75, 77]

for ipred in indices:
    for irep in range(0,1):
        true_range = parameter_matrix[ipred,0]
        true_smooth = parameter_matrix[ipred,1]
        field = fields[ipred,irep,:,:]
        visualize_calibrated_neural_likelihood_surface(field, possible_ranges, 
                                                       possible_smooths, true_range, true_smooth, irep, C)